In [66]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import collections
import math
import os
import random
import zipfile
import pyodbc
import sys
import re
import preprocessor
import tensorflow as tf

[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

['Microsoft Access Driver (*.mdb, *.accdb)']

In [67]:
def preprocess(paragraph):
    #sentences = tokenizer.tokenize(paragraph)
    sentences = string.split(paragraph, '\n') 
   
    return sentences

In [68]:
def GetPullRequestWords():
    words = []
    conn_str = (r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\PhD\Webtrace\WebTrace\WebTrace\WebTrace\PCIWebTrace\PCIWebTrace.mdb;')
    cnxn = pyodbc.connect(conn_str)
   
    crsr = cnxn.cursor()
    sql = "SELECT PullRequestDescription from PullRequest"
    crsr.execute(sql)
    # show the result
    result = crsr.fetchall();
    for item in result:      
        print(preprocessor.preprocessNoCamel(str(item)))
        words.append(preprocessor.preprocessNoCamel(str(item)))
    return words



def ReadData():
    Issues = []   
    IssuesLength = []
    PullRequests = []   
    PullRequestsLength = []
    VOCABULARY_SIZE = 100000
    
    conn_str = (r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\PhD\Webtrace\WebTrace\WebTrace\WebTrace\PCIWebTrace\PCIWebTrace.mdb;')
    cnxn = pyodbc.connect(conn_str)
   
    crsr = cnxn.cursor()
    issuesql = "SELECT Description from Issue"
    crsr.execute(issuesql)
    
    # show the result    
    result1 = crsr.fetchall();
    for item in result1: 
        value = preprocessor.preprocessNoCamel(str(item))
        Issues.append(value)
        IssuesLength.append(len(value))
 
    crsp = cnxn.cursor()
    pullrequestsql = "SELECT PullRequestDescription from PullRequest"
    crsp.execute(pullrequestsql)
    
    # show the result
    result2 = crsp.fetchall();
    for item in result2:      
        value = preprocessor.preprocessNoCamel(str(item))
        PullRequests.append(value)
        PullRequestsLength.append(len(value))                            
  
    return Issues,IssuesLength,PullRequests,PullRequestsLength

In [69]:
def build_dataset(words, n_words):
    word_counts = [['UNKNOWN', -1]]
    
    counter = collections.Counter(words)
    word_counts.extend(counter.most_common(n_words - 1))
 
    # Only the most common words are added to the dictionary
    dictionary = dict()
    
    for word, _ in word_counts:
        # The current length of the dictionary is the unique index of this word
        # added to the dictionary
        dictionary[word] = len(dictionary)
    
    word_indexes = list()
    
    unknown_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNKNOWN']
            unknown_count += 1
        
        word_indexes.append(index)
    
    # Count of unknown words
    word_counts[0][1] = unknown_count
    
    # Map from unique indexes to word
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    return word_counts, word_indexes, dictionary, reversed_dictionary

In [70]:
Issues,IssuesLength,PullRequests,PullRequestsLength = ReadData()
IssuesVol = []
for seq in Issues:
    print(seq)
    for word in seq:
            try:
                IssuesVol.append(word)
            except KeyError:
                IssuesVol.append(np.zeros(100))
    if len(IssuesVol) > 0:
        pass
    else:
        IssuesVol.append(np.zeros(100))        

[['typed', 'invalid', 'code', 'cursor', 'expression', 'show', 'cursor', 'typed', 'nusing', 'system', 'collection', 'generic', 'ninterface', 'ienumerable', 'int', 'void', 'add', 'int', 'nclass', 'void', 'new', 'ngot', 'stack', 'trace', 'tmicrosoft', 'codeanalysis', 'csharp', 'dll', 'microsoft', 'codeanalysis', 'csharp', 'binder', 'bindexpressioninternal', 'microsoft', 'codeanalysis', 'csharp', 'syntax', 'expressionsyntax', 'node', 'microsoft', 'codeanalysis', 'diagnosticbag', 'diagnostics', 'bool', 'invoked', 'bool', 'indexed', 'line', 'tc', 'tmicrosoft', 'codeanalysis', 'csharp', 'dll', 'microsoft', 'codeanalysis', 'csharp', 'binder', 'bindexpression', 'microsoft', 'codeanalysis', 'csharp', 'syntax', 'expressionsyntax', 'node', 'microsoft', 'codeanalysis', 'diagnosticbag', 'diagnostics', 'bool', 'invoked', 'bool', 'indexed', 'line', 'tc', 'tmicrosoft', 'codeanalysis', 'csharp', 'dll', 'microsoft', 'codeanalysis', 'csharp', 'binder', 'bindnamespaceortypeorexpression', 'microsoft', 'code

In [65]:
IssuesVol[0:10]
#VOCABULARY_SIZE = 100000
#word_counts, word_indexes, dictionary, reversed_dictionary = build_dataset(IssuesVol, VOCABULARY_SIZE)

[['typed',
  'invalid',
  'code',
  'cursor',
  'expression',
  'show',
  'cursor',
  'typed',
  'nusing',
  'system',
  'collection',
  'generic',
  'ninterface',
  'ienumerable',
  'int',
  'void',
  'add',
  'int',
  'nclass',
  'void',
  'new',
  'ngot',
  'stack',
  'trace',
  'tmicrosoft',
  'codeanalysis',
  'csharp',
  'dll',
  'microsoft',
  'codeanalysis',
  'csharp',
  'binder',
  'bindexpressioninternal',
  'microsoft',
  'codeanalysis',
  'csharp',
  'syntax',
  'expressionsyntax',
  'node',
  'microsoft',
  'codeanalysis',
  'diagnosticbag',
  'diagnostics',
  'bool',
  'invoked',
  'bool',
  'indexed',
  'line',
  'tc',
  'tmicrosoft',
  'codeanalysis',
  'csharp',
  'dll',
  'microsoft',
  'codeanalysis',
  'csharp',
  'binder',
  'bindexpression',
  'microsoft',
  'codeanalysis',
  'csharp',
  'syntax',
  'expressionsyntax',
  'node',
  'microsoft',
  'codeanalysis',
  'diagnosticbag',
  'diagnostics',
  'bool',
  'invoked',
  'bool',
  'indexed',
  'line',
  'tc',
  '